![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

# **MedicalTextGenerator**

This notebook will cover the different parameters and usages of `MedicalTextGenerator`.

**📖 Learning Objectives:**

1. Background: Understand the `MedicalTextGenerator` annotator.

2. Colab setup.

3. Become comfortable with using the different parameters of the annotator.

**🔗 Helpful Links:**

- Python Docs : [MedicalTextGenerator](https://nlp.johnsnowlabs.com/licensed/api/python/reference/autosummary/sparknlp_jsl/annotator/seq2seq/medical_text_generator/index.html#)

- Scala Docs: [MedicalTextGenerator](https://nlp.johnsnowlabs.com/licensed/api/com/johnsnowlabs/nlp/annotators/seq2seq/MedicalTextGenerator.html)

- For extended examples of usage, see [Spark NLP Workshop repository](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/healthcare-nlp/25.1.Medical_Text_Generation.ipynb).


## **📜 Background**

`MedicalTextGenerator` uses the basic **BioGPT** model to perform various tasks related to medical text abstraction.

With the help of this annotator, a user can provide a `prompt` and `context` and instruct the system to perform a specific task, such as explaining why a patient may have a particular disease or paraphrasing the context more directly.

In addition, this annotator can create a clinical note for a cancer patient using the given keywords or write medical texts based on introductory sentences.

The `BioGPT model` is trained on a large volume of medical data allowing it to identify and extract the most relevant information from the text provided.

## **🎬 Colab Setup**

In [ ]:
import os, json

from google.colab import files

if 'spark_jsl.json' not in os.listdir():
  license_keys = files.upload()
  os.rename(list(license_keys.keys())[0], 'spark_jsl.json')

with open('spark_jsl.json') as f:
    license_keys = json.load(f)

locals().update(license_keys)
os.environ.update(license_keys)


In [ ]:
!pip install -q johnsnowlabs

In [ ]:
from johnsnowlabs import nlp, medical
nlp.install()


In [ ]:
from johnsnowlabs import nlp, medical

spark = nlp.start()

In [ ]:
spark

In [ ]:
def f(text):
  import textwrap
  text = textwrap.fill(text, width=120)
  return  '\n'+text

## **🖨️ Input/Output Annotation Types**



- Input:  `DOCUMENT`  

- Output:  `CHUNK`  

## **🔎 Parameters**

`caseSensitive`: whether to ignore case in tokens for embeddings matching; default: True.

`maxNewTokens`: maximum number of new tokens to be generated; default: 30.

`doSample`: whether or not to use sampling; use greedy decoding otherwise; default: False.

`topK`: the number of highest probability vocabulary tokens to consider; default: 1.

`stopAtEos`: Stop text generation when the end-of-sentence token is encountered; default: True.

`maxContextLength`: maximum length of the context text; default: 1024.

`maxTextLength`: maximum text length to process; default:512.

`noRepeatNgramSize`: if set to int > 0, all ngrams of that size can only occur once; default: 0.

`customPrompt`: different formats may be assigned like "question: {DOCUMENT} answer:" or "INPUT: {DOCUMENT} OUTPUT:" or else.

# ✍  Explaining medical.TextGenerator with an Example

## **💻nlp.Pipeline**

In [ ]:
document_assembler = nlp.DocumentAssembler()\
    .setInputCol("prompt")\
    .setOutputCol("document_prompt")

med_text_generator  = medical.TextGenerator.pretrained("text_generator_biomedical_biogpt_base", "en", "clinical/models")\
    .setInputCols("document_prompt")\
    .setOutputCol("answer")\
    .setCaseSensitive(True)\
    .setMaxNewTokens(256)\
    .setDoSample(True)\
    .setTopK(3)\
    .setRandomSeed(40)\
    .setStopAtEos(True)

pipeline = nlp.Pipeline(stages=[document_assembler, med_text_generator])
data = spark.createDataFrame([['Covid 19 is']]).toDF("prompt")

result = pipeline.fit(data).transform(data)

In [ ]:
# let's see the quick result

res = result.select("answer.result").toPandas().iloc[0][0]
res

['Covid 19 is an emerging pandemic with an unprecedented number and variety. ( 1,2 &#93; The pandemic is caused mainly due the SARS CoV 2 ( SARS CoV 2, the virus causing the coronavirus ), a virus which is a novel member in Coronaviral family and is responsible of a highly transmissible and lethal respiratory illness known to cause severe acute and critical illnesses in human beings &#91; 1 - 4, 1 - 6, 7 &#93;, which has resulted into the global health crisis of COVID - 2019 &#91; 8 - 11, 1 - 4 &#93;, which is currently spreading across all countries in Asia and Africa. ( 1,2 - 5 ) In the current study we report a novel case in which the COVID 19 was confirmed in an Indian male, aged 25, with no past history or clinical evidence for other viral or autoimmune disease, and the case is presented here for the rarity. ( 6 - 9, 10 )']

In [ ]:
text = ["COVID-19",
        "SARS-CoV-2",
        "Asthma is a chronic respiratory disease characterized by"]
model = pipeline.fit(data)

light_model = nlp.LightPipeline(model)
light_result = light_model.annotate(text)

for i in range(len(text)):
  print(f"document_prompt ➤ {f(light_result[i]['document_prompt'][0])}\n answer ➤ {f(light_result[i]['answer'][0])} \n\n")

document_prompt ➤ 
COVID-19
 answer ➤ 
COVID - 19 and diabetes: a review of current literature and implications. ( 1 - 4. 


document_prompt ➤ 
SARS-CoV-2
 answer ➤ 
SARS - CoV - 2 is the cause for COVID 19. ( ABSTRACT 


document_prompt ➤ 
Asthma is a chronic respiratory disease characterized by
 answer ➤ 
Asthma is a chronic respiratory disease characterized by airway hyperresponsiveness, airway remodeling and airway
inflammation. ( ABSTRACT 




In [ ]:
med_text_generator.extractParamMap()

{Param(parent='MedicalTextGenerator_9430e26a418f', name='batchSize', doc='Size of every batch'): 4,
 Param(parent='MedicalTextGenerator_9430e26a418f', name='customPrompt', doc='Custom prompt template'): '',
 Param(parent='MedicalTextGenerator_9430e26a418f', name='doSample', doc='Whether or not to use sampling; use greedy decoding otherwise'): True,
 Param(parent='MedicalTextGenerator_9430e26a418f', name='ignoreTokenIds', doc="A list of token ids which are ignored in the decoder's output"): JavaObject id=o90,
 Param(parent='MedicalTextGenerator_9430e26a418f', name='lazyAnnotator', doc='Whether this AnnotatorModel acts as lazy in RecursivePipelines'): False,
 Param(parent='MedicalTextGenerator_9430e26a418f', name='maxContextLength', doc='Maximum length of the context text'): 1024,
 Param(parent='MedicalTextGenerator_9430e26a418f', name='maxNewTokens', doc='Maximum number of new tokens to be generated'): 256,
 Param(parent='MedicalTextGenerator_9430e26a418f', name='maxTextLength', doc='Ma

### ▶ `caseSensitive` :

Whether to ignore case sensitivity in tokens.

In [ ]:
%%time

CaseSensitive = [False, True]
text = ["Asthma is a chronic respiratory disease characterized by"]

for cs in CaseSensitive:
  med_text_generator  = medical.TextGenerator.pretrained("text_generator_biomedical_biogpt_base", "en", "clinical/models")\
    .setInputCols("document_prompt")\
    .setOutputCol("answer")\
    .setCaseSensitive(cs)

  pipeline = nlp.Pipeline(stages=[document_assembler, med_text_generator])

  model = pipeline.fit(data)

  light_model = nlp.LightPipeline(model)
  light_result = light_model.annotate(text)

  print('■'*120)
  print('CaseSensitive = ',cs)
  for i in range(len(text)):
    print(f"document_prompt ➤ {f(light_result[i]['document_prompt'][0])}\n answer ➤ {f(light_result[i]['answer'][0])} \n\n")

text_generator_biomedical_biogpt_base download started this may take some time.
[OK!]
■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■
CaseSensitive =  False
document_prompt ➤ 
Asthma is a chronic respiratory disease characterized by
 answer ➤ 
asthma is a chronic respiratory disease characterized by airway hyperresponsiveness, airway remodeling and airway
hyperresponsiveness ( AHT, airway hyperresponsiveness to non specific bronchoconstrictor agents such methacholine and
bradykinin, which are thought by many investigators as the main factors in asthma ), which are the hallmarks for asthma
pathogenesis and progression to asthma. ( ABSTRACT 


text_generator_biomedical_biogpt_base download started this may take some time.
[OK!]
■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■
CaseSensitive =  True
document_prompt ➤ 
Asthma is a chronic respirato

### ▶ `maxNewTokens`:
Maximum number of new tokens to be generated.


In [ ]:
tokenNumber = [1,30]
text = ["Asthma is a chronic respiratory disease characterized by"]
for tn in  tokenNumber:
  med_text_generator  = medical.TextGenerator.pretrained("text_generator_biomedical_biogpt_base", "en", "clinical/models")\
      .setInputCols("document_prompt")\
      .setOutputCol("answer")\
      .setCaseSensitive(True)\
      .setMaxNewTokens(tn)

  pipeline = nlp.Pipeline(stages=[document_assembler, med_text_generator])
  model = pipeline.fit(data)

  light_model = nlp.LightPipeline(model)
  light_result = light_model.annotate(text)

  print('■'*120)
  print('setMaxNewTokens = ', tn)

  for i in range(len(text)):
    print(f"document_prompt ➤ {f(light_result[i]['document_prompt'][0])}\n answer ➤ {f(light_result[i]['answer'][0])} \n\n")

text_generator_biomedical_biogpt_base download started this may take some time.
[OK!]
■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■
setMaxNewTokens =  1
document_prompt ➤ 
Asthma is a chronic respiratory disease characterized by
 answer ➤ 
Asthma is a chronic respiratory disease characterized by airway 


text_generator_biomedical_biogpt_base download started this may take some time.
[OK!]
■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■
setMaxNewTokens =  30
document_prompt ➤ 
Asthma is a chronic respiratory disease characterized by
 answer ➤ 
Asthma is a chronic respiratory disease characterized by airway hyperresponsiveness, airway remodeling and airway
obstruction, and it has a significant negative effect in quality - adjusted survival and health care expenditure in
patients. ( ABSTRACT 




### ▶ `doSample`:
Sets whether or not to use sampling.

In [ ]:
setDoSample = [True, False]
text = ["Asthma is a chronic respiratory disease characterized by"]
med_text_generator  = medical.TextGenerator.pretrained("text_generator_biomedical_biogpt_base", "en", "clinical/models")\
      .setInputCols("document_prompt")\
      .setOutputCol("answer")\
      .setCaseSensitive(True)\
      .setMaxNewTokens(50)

for ds in setDoSample:
  med_text_generator  = med_text_generator.setDoSample(ds)

  pipeline = nlp.Pipeline(stages=[document_assembler, med_text_generator])
  model = pipeline.fit(data)

  light_model = nlp.LightPipeline(model)
  light_result = light_model.annotate(text)

  print('■'*120)
  print('setDoSample = ', ds)

  for i in range(len(text)):
    print(f"document_prompt ➤ {f(light_result[i]['document_prompt'][0])}\n\n\n answer ➤ {f(light_result[i]['answer'][0])} \n")

text_generator_biomedical_biogpt_base download started this may take some time.
[OK!]
■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■
setDoSample =  True
document_prompt ➤ 
Asthma is a chronic respiratory disease characterized by


 answer ➤ 
Asthma is a chronic respiratory disease characterized by airway hyperresponsiveness, airway remodeling and airway
obstruction, and it has a significant negative effect in quality - adjusted survival and health care expenditure in
patients. ( ABSTRACT 

■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■
setDoSample =  False
document_prompt ➤ 
Asthma is a chronic respiratory disease characterized by


 answer ➤ 
Asthma is a chronic respiratory disease characterized by reversible airway obstruction. 



### ▶`topK`:
Number of highest probability vocabulary tokens to consider.



In [ ]:
text = ["The commont treatment method for diabetes are", "COVID-19 is"]
med_text_generator  = medical.TextGenerator.pretrained("text_generator_biomedical_biogpt_base", "en", "clinical/models")\
    .setInputCols("document_prompt")\
    .setOutputCol("answer")\
    .setCaseSensitive(True)\
    .setMaxNewTokens(25)\
    .setDoSample(True)

for topk in [1,5]:
  med_text_generator = med_text_generator.setTopK(topk)

  pipeline = nlp.Pipeline(stages=[document_assembler, med_text_generator])
  model = pipeline.fit(data)


  light_model = nlp.LightPipeline(model)
  light_result = light_model.annotate(text)

  print('■'*120)
  print('setTopK = ', topk)

  for i in range(len(text)):
    print(f"document_prompt ➤ {f(light_result[i]['document_prompt'][0])}\n\n\n answer ➤ {f(light_result[i]['answer'][0])} \n")
    print('-'*120)

text_generator_biomedical_biogpt_base download started this may take some time.
[OK!]
■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■
setTopK =  1
document_prompt ➤ 
The commont treatment method for diabetes are


 answer ➤ 
The commont treatment method for diabetes are insulin injections and oral hypoglycemic agents. 

------------------------------------------------------------------------------------------------------------------------
document_prompt ➤ 
COVID-19 is


 answer ➤ 
COVID - 19 is a global pandemic. 

------------------------------------------------------------------------------------------------------------------------
■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■
setTopK =  5
document_prompt ➤ 
The commont treatment method for diabetes are


 answer ➤ 
The commont treatment method for diabetes are oral medications and lifes

### ▶`stopAtEos`:
Stop text generation when the end-of-sentence token is encountered.

In [ ]:
text = ["COVID-19 is"]
med_text_generator  = medical.TextGenerator.pretrained("text_generator_biomedical_biogpt_base", "en", "clinical/models")\
  .setInputCols("document_prompt")\
  .setOutputCol("answer")\
  .setCaseSensitive(True)\
  .setMaxNewTokens(25)\
  .setDoSample(False)\
  .setTopK(5)

for opt in [True, False]:
  med_text_generator = med_text_generator.setStopAtEos(opt)
  pipeline = nlp.Pipeline(stages=[document_assembler, med_text_generator])
  model = pipeline.fit(data)

  light_model = nlp.LightPipeline(model)
  light_result = light_model.annotate(text)

  print('■'*120)
  print('setStopAtEos = ', opt)

  for i in range(len(text)):
    print(f"document_prompt ➤ {f(light_result[i]['document_prompt'][0])}\n\n\n answer ➤ {f(light_result[i]['answer'][0])} \n")

text_generator_biomedical_biogpt_base download started this may take some time.
[OK!]
■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■
setStopAtEos =  True
document_prompt ➤ 
COVID-19 is


 answer ➤ 
COVID - 19 is a global pandemic. 

■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■
setStopAtEos =  False
document_prompt ➤ 
COVID-19 is


 answer ➤ 
COVID - 19 is a global pandemic. The World Health Organization declared COVID - 19 a pandemic on March 11, 2020. The
disease is 



### ▶`maxContextLength`:
Maximum length of the context text; default: 1024.

In [ ]:
text = ["Asthma is a chronic respiratory disease characterized by"]

med_text_generator  = medical.TextGenerator.pretrained("text_generator_biomedical_biogpt_base", "en", "clinical/models")\
  .setInputCols("document_prompt")\
  .setOutputCol("answer")\
  .setCaseSensitive(True)\
  .setMaxNewTokens(512)\
  .setDoSample(True)\
  .setTopK(50)\
  .setStopAtEos(True)

for opt in [2, 50]:
  med_text_generator = med_text_generator.setMaxContextLength(opt)
  pipeline = nlp.Pipeline(stages=[document_assembler, med_text_generator])
  model = pipeline.fit(data)

  light_model = nlp.LightPipeline(model)
  light_result = light_model.annotate(text)

  print('■'*120)
  print('setMaxContextLength = ', opt)

  for i in range(len(text)):
    print(f"document_prompt ➤ {f(light_result[i]['document_prompt'][0])}\n\n\n answer ➤ {f(light_result[i]['answer'][0])} \n")

text_generator_biomedical_biogpt_base download started this may take some time.
[OK!]
■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■
setMaxContextLength =  2
document_prompt ➤ 
Asthma is a chronic respiratory disease characterized by


 answer ➤ 
Asthma is a chronic respiratory disease characterized by inflammatory reaction due airway mucosa that mainly targets on
large smooth pulmonary arterioles due changes during gestation such inflammation has different inflammatory reactions
among its exacerbations due gestational periods as birth ( labor stage B &#91; N group was  29 w.o in B. We aim with
finding possible predictive biomarker such with biomarkers with changes as different clinical outcomes between antenatal
exacerbations vs postpartum visits at our Institute as birth, or first - day hospitalization / other postpartum relapses
as in preeclampsia vs pregnancy / parturition complicated asthmatics from previous

### ▶`maxTextLength`
Maximum text length to process; default:512.

In [ ]:
text = ["Asthma is a chronic respiratory disease characterized by"]

med_text_generator  = medical.TextGenerator.pretrained("text_generator_biomedical_biogpt_base", "en", "clinical/models")\
  .setInputCols("document_prompt")\
  .setOutputCol("answer")\
  .setCaseSensitive(True)\
  .setMaxNewTokens(512)\
  .setDoSample(False)\
  .setTopK(1)\
  .setStopAtEos(True)

for opt in [2, 50]:
  med_text_generator = med_text_generator.setMaxTextLength(opt)
  pipeline = nlp.Pipeline(stages=[document_assembler, med_text_generator])
  model = pipeline.fit(data)

  light_model = nlp.LightPipeline(model)
  light_result = light_model.annotate(text)

  print('■'*120)
  print('setMaxTextLength = ', opt,'\n')

  for i in range(len(text)):
    print(f"document_prompt ➤ {f(light_result[i]['document_prompt'][0])}\n answer ➤ {f(light_result[i]['answer'][0])} \n")

text_generator_biomedical_biogpt_base download started this may take some time.
[OK!]
■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■
setMaxTextLength =  2 

document_prompt ➤ 
Asthma is a chronic respiratory disease characterized by
 answer ➤ 
Asthma is a chronic respiratory disease characterized by reversible 

■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■
setMaxTextLength =  50 

document_prompt ➤ 
Asthma is a chronic respiratory disease characterized by
 answer ➤ 
Asthma is a chronic respiratory disease characterized by reversible airway obstruction. 



### ▶`noRepeatNgramSize`

If set to int > 0, all ngrams of that size can only occur once, Default: 0



In [ ]:
text = ["Asthma is a chronic respiratory disease characterized by"]

med_text_generator  = medical.TextGenerator.pretrained("text_generator_biomedical_biogpt_base", "en", "clinical/models")\
  .setInputCols("document_prompt")\
  .setOutputCol("answer")\
  .setCaseSensitive(True)\
  .setMaxNewTokens(512)\
  .setDoSample(False)\
  .setTopK(1)\
  .setStopAtEos(True)

for opt in [0,1 ]:
  med_text_generator = med_text_generator.setNoRepeatNgramSize(opt)
  pipeline = nlp.Pipeline(stages=[document_assembler, med_text_generator])
  model = pipeline.fit(data)

  light_model = nlp.LightPipeline(model)
  light_result = light_model.annotate(text)

  print('■'*120)
  print('setNoRepeatNgramSize = ', opt,'\n')

  for i in range(len(text)):
    print(f"document_prompt ➤ {f(light_result[i]['document_prompt'][0])}\n answer ➤ {f(light_result[i]['answer'][0])} \n")

text_generator_biomedical_biogpt_base download started this may take some time.
[OK!]
■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■
setNoRepeatNgramSize =  0 

document_prompt ➤ 
Asthma is a chronic respiratory disease characterized by
 answer ➤ 
Asthma is a chronic respiratory disease characterized by reversible airway obstruction. 

■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■
setNoRepeatNgramSize =  1 

document_prompt ➤ 
Asthma is a chronic respiratory disease characterized by
 answer ➤ 
Asthma is a chronic respiratory disease characterized by reversible airway obstruction. ( ABSTRACT TRUNCATED AT 250
WORDS ) The pathogenesis of asthma involves both immunologic and nonimmune mechanisms, including the release or action
on inflammatory cells such as mast cell tryptase from activated eosinophils in response to 



### ▶`customPrompt`


Different formats may be assigned like "question: {DOCUMENT} answer:" or "INPUT: {DOCUMENT} OUTPUT:" or else.


In [ ]:
text = ["What is the difference between melanoma and sarcoma?"]

med_text_generator  = medical.TextGenerator.pretrained("text_generator_biomedical_biogpt_base", "en", "clinical/models")\
  .setInputCols("document_prompt")\
  .setOutputCol("answer")\
  .setCaseSensitive(True)\
  .setMaxNewTokens(512)\
  .setDoSample(False)\
  .setTopK(1)\
  .setStopAtEos(True)\
  .setNoRepeatNgramSize(1)

for opt in ["question: {DOCUMENT} answer:", "INPUT: {DOCUMENT} OUTPUT:",""]:

  med_text_generator = med_text_generator.setCustomPrompt(opt)
  pipeline = nlp.Pipeline(stages=[document_assembler, med_text_generator])
  model = pipeline.fit(data)

  light_model = nlp.LightPipeline(model)
  light_result = light_model.annotate(text)

  print('■'*120)

  for i in range(len(text)):
    print(f"{f(light_result[i]['answer'][0])} \n")

text_generator_biomedical_biogpt_base download started this may take some time.
[OK!]
■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■

question: What is the difference between melanoma and sarcoma? answer: A systematic review. ( 1 ) The authors searched
MEDLINE, EMBASE Classic + Embase databases from inception to June 2017 for studies comparing survival outcomes of
patients with malignant melanomas versus sarcomas in adults 

■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■

input: What is the difference between melanoma and sarcoma? output: A review of current literature. ( 1 ) Melanoma, a
rare cancer with an incidence rate that has been increasing over time in many countries worldwide; however it remains
one which accounts for approximately 

■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■